<a href="https://colab.research.google.com/github/JoyBoy-00/NLP-and-Flask-Text-Summarize-Web-App/blob/main/modeltextsummary1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh # download ollama api
from IPython.display import clear_output

# Create a Python script to start the Ollama API server in a separate thread

import os
import threading
import subprocess
import requests
import json

def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpci3 pci.ids
The following NEW packages will be installed:
  libpci3 pci.ids pciutils
0 upgraded, 3 newly installed, 0 to remove and 29 not upgraded.
Need to get 343 kB of archives.
After this operation, 1,581 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 pci.ids all 0.0~2022.01.22-1 [251 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpci3 amd64 1:3.7.0-6 [28.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 pciutils amd64 1:3.7.0-6 [63.6 kB]
Fetched 343 kB in 1s (317 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: falling back to frontend: Readline
debconf: unable to initializ

In [ ]:
!pip install sentence-transformers
from IPython.display import clear_output
!ollama pull llama3.2:1b
clear_output()
!pip install -U lightrag[ollama]
from IPython.display import clear_output, Markdown, display
from lightrag.core.generator import Generator
from lightrag.core.component import Component
from lightrag.core.model_client import ModelClient
from lightrag.components.model_client import OllamaClient
from sentence_transformers import SentenceTransformer, util
import torch

# Initialize SBERT model
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')

# Define the QA template
qa_template = r"""
You are a helpful assistant.

User: {{input_str}}
You:"""

class SBERTQA(Component):
    def __init__(self, model_client: ModelClient, model_kwargs: dict):
        super().__init__()
        self.generator = Generator(
            model_client=model_client,
            model_kwargs=model_kwargs,
            template=qa_template,
        )
        self.sbert_model = SentenceTransformer('all-MiniLM-L6-v2')

    def call(self, input: dict) -> str:
        # Generate SBERT embeddings for the input text
        input_text = str(input)
        embeddings = self.sbert_model.encode(input_text, convert_to_tensor=True)

        # Use the embeddings to generate a summary
        summary = self.generator.call({"input_str": input_text})
        return summary

    async def acall(self, input: dict) -> str:
        # Generate SBERT embeddings for the input text
        input_text = str(input)
        embeddings = self.sbert_model.encode(input_text, convert_to_tensor=True)

        # Use the embeddings to generate a summary
        summary = await self.generator.acall({"input_str": input_text})
        return summary

# Initialize the model
model = {
    "model_client": OllamaClient(),
    "model_kwargs": {"model": "llama3.2:1b"}
}

# Create the SBERTQA component
qa = SBERTQA(**model)

# Generate a summary
input_text = "Summarize in 200 words What is the meaning of life?"
output = qa.call(input_text)

# Display the summary
display(Markdown(f"**Answer:** {output.data}"))

**Answer:** The question of the meaning of life is one of the most profound and complex questions humanity has ever pondered. There is no straightforward answer, as it depends on individual perspectives, beliefs, and values. However, various philosophical, religious, and cultural theories attempt to provide insights.

Some possible interpretations include:

- Existentialism: The idea that life has no inherent meaning, but individuals must create their own purpose through choices and actions.
- Stoicism: The pursuit of virtue, self-control, and inner strength as a means to find meaning in life.
- Optimism: Believing that life's meaning can be discovered through personal growth, relationships, and achievements.
- Nihilism: The rejection of all inherent meaning or value, leading individuals to focus on individual freedom and choice.

Ultimately, the meaning of life may be unique to each person, shaped by their experiences, emotions, and perspectives. While some find purpose in relationships, work, or personal growth, others may derive meaning from spirituality, art, or a sense of transcendence. The search for meaning is a lifelong journey, with no definitive answer, but rather a continuous exploration of one's values and passions.

In [ ]:
# prompt: clone repo in github

!git clone https://github.com/username/repository.git


Cloning into 'NLP-and-Flask-Text-Summarize-Web-App'...
fatal: could not read Username for 'https://github.com': No such device or address
